In [1]:
import subprocess
import shlex
import json
from glob import glob
from tqdm import tqdm_notebook as tqdm
import pandas as pd

In [2]:
df = pd.read_csv('./data_models/events_joined_total_08.04.2021.csv', index_col=0)
cat_cols = ['city', 'country', 'is_fight_night', 'weightCategory.id', 
              'f1_country', 'f2_country', 'f1_city', 'f2_city']

df[cat_cols] = df[cat_cols].fillna('unknown')
df = df[(df['f1_count_of_fights'] >= 2) & (df['f2_count_of_fights'] >= 2)]

df['eventDate.date'] = pd.to_datetime(df['eventDate.date'])
df['weightCategory.id'] = df['weightCategory.id'].astype('object')
df  = df[(df['eventDate.date'].dt.year >= 2020) &
        ((df['type']=='straight'))]
df = df[~df['odds_difference'].isna()]

df

,city,country,duration,eventDate.date,eventDate.timezone,fighterId_1,fighterId_2,id,name,rounds,...,odds_difference,age_difference,height_difference,armSpan_difference,legSwing_difference,isHomeCity_difference,isHomeCountry_difference,isHomeTimezone_difference,winner,is_fight_night
6359,Las Vegas,USA,40.0,2020-01-18,Europe/Berlin,347,211,39796,UFC 246,1.0,...,2.32,5.0,10.16,-2.54,1.27,0.0,1.0,0.0,False,False
6360,Las Vegas,USA,106.0,2020-01-18,Europe/Berlin,292,404,39804,UFC 246,2.0,...,1.47,-3.0,2.54,-5.08,0.00,0.0,1.0,0.0,False,False
6361,Las Vegas,USA,300.0,2020-01-18,Europe/Berlin,562,427,39805,UFC 246,3.0,...,0.39,-7.0,-2.54,-5.08,-2.54,0.0,0.0,0.0,False,False
6362,Las Vegas,USA,300.0,2020-01-18,Europe/Berlin,521,407,39807,UFC 246,3.0,...,-0.47,-3.0,-5.08,-7.62,0.10,0.0,-1.0,0.0,True,False
6363,Las Vegas,USA,300.0,2020-01-18,Europe/Berlin,525,233,39820,UFC 246,3.0,...,-5.68,-16.0,-5.08,-10.16,5.18,0.0,0.0,0.0,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6870,Las Vegas,USA,300.0,2021-02-27,Europe/Berlin,533,496,40495,UFC Fight Night,3.0,...,0.39,3.0,5.08,-7.62,-2.54,0.0,-1.0,0.0,True,True
6871,Las Vegas,USA,300.0,2021-02-27,Europe/Berlin,135,208,40496,UFC Fight Night,3.0,...,1.31,13.0,-2.54,-5.08,-7.72,0.0,0.0,0.0,True,True
6872,Las Vegas,USA,300.0,2021-02-27,Europe/Berlin,421,668,40497,UFC Fight Night,3.0,...,-1.38,3.0,0.00,5.08,-1.37,0.0,1.0,0.0,False,True
6875,Las Vegas,USA,208.0,2021-03-06,Europe/Berlin,2073,246,40452,UFC 259,3.0,...,-1.27,2.0,-5.08,-10.16,-7.52,0.0,1.0,0.0,True,False


In [3]:
indexes = df.index
f1_id, f2_id, f1_odd, f2_odd, weightCategory_id, city, country, eventDate, eventName, eventTimezone = \
    df.loc[indexes[0], ['fighterId_1', 'fighterId_2', 'f1_odds', 'f2_odds', 'weightCategory.id',
                        'city', 'country', 'eventDate.date', 'name', 'timezone']]

f1_id, f2_id, f1_odd, f2_odd, weightCategory_id, city, country, eventDate, eventName, eventTimezone

(347,
 211,
 3.61,
 1.29,
 14,
 'Las Vegas',
 'USA',
 Timestamp('2020-01-18 00:00:00'),
 'UFC 246',
 'America/Los_Angeles')

In [5]:
Curl = f"""curl -X 'GET' \
  'http://127.0.0.1:8001/predict_fight?f1_id={f1_id}&f2_id={f2_id}&f1_odd={f1_odd}&f2_odd={f2_odd}&\
weightCategory_id={weightCategory_id}&\
city={city.replace(' ', '%20')}&\
country={country}&\
event_date={str(eventDate)[:10]}&\
event_name={eventName.replace(' ', '%20')}&\
time_zone={eventTimezone}' \
  -H 'accept: application/json'"""

args = shlex.split(Curl)
process = subprocess.Popen(args, shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = process.communicate()
response = json.loads(stdout.decode("UTF-8"))
response

{'y_proba_catboost_v1_0': 0.4168568594249422,
 'y_proba_catboost_v1_1': 0.31852239037247054}